# Lab 6: Building A RAG Agent

## Setup Components

In [ ]:
## Load Environment Table
from dotenv import load_dotenv
load_dotenv(override=True)  # take environment variables

In [ ]:
# Load large language model 
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4.1-nano", model_provider="openai")

In [ ]:
%pip install langchain_huggingface

In [ ]:
# Selecting Embedding model
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

from langchain_huggingface import HuggingFaceEmbeddings
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
# embedding_model = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-Qwen2-7B-instruct")

In [ ]:
#from langchain_openai import OpenAIEmbeddings
#embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# Initiate Vector Code
# In-memory 

from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embedding_model)

## Document Loaders

In [ ]:
%pip install -qU pypdf

In [ ]:
##
file_path = "./docs/Tentang Dexa Medica.pdf"

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

In [ ]:
docs = loader.load()
docs

## Splitting

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100
)

all_splits = text_splitter.split_documents(docs)

len(all_splits)

In [ ]:
for split in all_splits:
    print(split)

## Embedding and Indexing

In [ ]:
vector_store.add_documents(all_splits)

In [ ]:
vector_store.similarity_search("siapa pendiri Dexa Medica")

## Defining Nodes

In [ ]:
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=5)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
from langgraph.graph import MessagesState, StateGraph
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Keep the answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [ ]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder = StateGraph(MessagesState)
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
input_message = "Hello"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "1"}}

input_message = "Siapa pendiri Dexa Medica?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "dimana Dexa Medica didirikan?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "Sebutkan 4 kompetensi inti yang dimiliki Dexa Medica."

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "Apa saja kontribusi yang sudah dilakukan oleh Dexa Medica?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "Carikan informasi tentang Dexa Medica. Meliputi Motto, lokasi, dan pendiri."

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "2"}}

input_message = "sebutkan visi dan misi Dexa Medica"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

### Check Similarity Search

In [ ]:
vector_store.similarity_search("sebutkan visi dan misi Dexa Medica")

# END